<a href="https://colab.research.google.com/github/dtabuena/Workshop/blob/main/Trodes/Exporting_Trodes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [43]:
import os
import subprocess

In [44]:
reccording_source_dir = r'\\hive.gladstone.internal\Huang-LFP\TabuenaLFP\Test_Emily'
os.chdir(reccording_source_dir)

dot_dat_export_dir = r'\\hive.gladstone.internal\Huang-LFP\TabuenaLFP\TRODE_EXPORTS'
os.chdir(dot_dat_export_dir)

rec_list=list()
for root, dirs, files in os.walk(reccording_source_dir):
    for f in files:
        if '.rec' in f:
            rec_list.append(os.path.join(root,f))
print(rec_list)


export_func_location = 'C:/Users/dennis.tabuena/Desktop/_SPIKE_GADGETS/Trodes_2-5-1_Windows64/Trodes_2-5-1_Windows64/trodesexport.exe'
filename = rec_list[0]

['\\\\hive.gladstone.internal\\Huang-LFP\\TabuenaLFP\\Test_Emily\\AngelfoodCake01\\AngelfoodCake01.rec']


In [ ]:
""" HELP """
subprocess.run(export_func_location+' -h')

CompletedProcess(args='C:/Users/dennis.tabuena/Desktop/_SPIKE_GADGETS/Trodes_2-5-1_Windows64/Trodes_2-5-1_Windows64/trodesexport.exe -h', returncode=0)

In [ ]:
raw_dir = r'\\hive.gladstone.internal\Huang-LFP\TabuenaLFP\TRODE_EXPORTS\RAW_DATA'
full_cmd_raw = ' '.join([export_func_location, '-raw',
                    '-rec ', filename,
                    '-userawrefs', str(0),
                    '-outputdirectory', raw_dir])
print(full_cmd_raw)
subprocess.run(full_cmd_raw)

C:/Users/dennis.tabuena/Desktop/_SPIKE_GADGETS/Trodes_2-5-1_Windows64/Trodes_2-5-1_Windows64/trodesexport.exe -raw -rec  \\hive.gladstone.internal\Huang-LFP\TabuenaLFP\Test_Emily\AngelfoodCake01\AngelfoodCake01.rec -userawrefs 0 -outputdirectory \\hive.gladstone.internal\Huang-LFP\TabuenaLFP\TRODE_EXPORTS\RAW_DATA


CompletedProcess(args='C:/Users/dennis.tabuena/Desktop/_SPIKE_GADGETS/Trodes_2-5-1_Windows64/Trodes_2-5-1_Windows64/trodesexport.exe -raw -rec  \\\\hive.gladstone.internal\\Huang-LFP\\TabuenaLFP\\Test_Emily\\AngelfoodCake01\\AngelfoodCake01.rec -userawrefs 0 -outputdirectory \\\\hive.gladstone.internal\\Huang-LFP\\TabuenaLFP\\TRODE_EXPORTS\\RAW_DATA', returncode=0)

In [ ]:
lfp_dir = r'\\hive.gladstone.internal\Huang-LFP\TabuenaLFP\TRODE_EXPORTS\LFP_DATA'
full_cmd_lfp = ' '.join([export_func_location, '-lfp',
                         '-rec ', filename,
                         '-lfpoutputrate', str(5000),
                         '-uselfprefs', str(0),
                         '-uselfpfilters', str(0),
                         '-outputdirectory', lfp_dir])
report = subprocess.run(full_cmd_lfp)
print(report)

CompletedProcess(args='C:/Users/dennis.tabuena/Desktop/_SPIKE_GADGETS/Trodes_2-5-1_Windows64/Trodes_2-5-1_Windows64/trodesexport.exe -lfp -rec  \\\\hive.gladstone.internal\\Huang-LFP\\TabuenaLFP\\Test_Emily\\AngelfoodCake01\\AngelfoodCake01.rec -lfpoutputrate 5000 -uselfprefs 0 -uselfpfilters 0 -outputdirectory \\\\hive.gladstone.internal\\Huang-LFP\\TabuenaLFP\\TRODE_EXPORTS\\LFP_DATA', returncode=0)


In [ ]:
spike_dir = r'\\hive.gladstone.internal\Huang-LFP\TabuenaLFP\TRODE_EXPORTS\SPIKE_DATA'
full_cmd_spikeband = ' '.join([export_func_location, '-spikeband',
                         '-rec ', filename,
                         '-spikehighpass', str(600),
                         '-spikelowpass', str(6000),
                         '-usespikerefs', str(0),
                         '-usespikefilters', str(0),
                         '-outputdirectory', spike_dir])
report = subprocess.run(full_cmd_spikeband)
print(report)

CompletedProcess(args='C:/Users/dennis.tabuena/Desktop/_SPIKE_GADGETS/Trodes_2-5-1_Windows64/Trodes_2-5-1_Windows64/trodesexport.exe -spikeband -rec  \\\\hive.gladstone.internal\\Huang-LFP\\TabuenaLFP\\Test_Emily\\AngelfoodCake01\\AngelfoodCake01.rec -spikehighpass 600 -spikelowpass 6000 -usespikerefs 0 -usespikefilters 0 -outputdirectory \\\\hive.gladstone.internal\\Huang-LFP\\TabuenaLFP\\TRODE_EXPORTS', returncode=0)

In [ ]:
raw_dir = r'\\hive.gladstone.internal\Huang-LFP\TabuenaLFP\TRODE_EXPORTS\RAW_DATA'
full_cmd_raw = ' '.join([export_func_location, '-raw',
                         '-rec ', filename,
                         '-spikehighpass', str(600),
                         '-spikelowpass', str(6000),
                         '-usespikerefs', str(0),
                         '-usespikefilters', str(0),
                         '-outputdirectory', raw_dir])
report = subprocess.run(raw_dir)
print(report)

In [ ]:
os.chdir('C:/Users/dennis.tabuena/Desktop/_SPIKE_GADGETS/Trodes_2-5-1_Windows64/Trodes_2-5-1_Windows64/AngelfoodCake01.raw')
x = readTrodesExtractedDataFile('AngelfoodCake01.raw_group0.dat')

In [ ]:
#!/usr/bin/python
#Requires numpy to be installed. re and sys are defaults for python
#assumes python 2.7
import numpy as np
import re
from sys import argv
# Main function
def readTrodesExtractedDataFile(filename):
    with open(filename, 'rb') as f:
        # Check if first line is start of settings block
        if f.readline().decode('ascii').strip() != '<Start settings>':
            raise Exception("Settings format not supported")
        fields = True
        fieldsText = {}
        for line in f:
            # Read through block of settings
            if(fields):
                line = line.decode('ascii').strip()
                # filling in fields dict
                if line != '<End settings>':
                    vals = line.split(': ')
                    fieldsText.update({vals[0].lower(): vals[1]})
                # End of settings block, signal end of fields
                else:
                    fields = False
                    dt = parseFields(fieldsText['fields'])
                    fieldsText['data'] = np.zeros([1], dtype = dt)
                    break
        # Reads rest of file at once, using dtype format generated by parseFields()
        dt = parseFields(fieldsText['fields'])
        data = np.fromfile(f, dt)
        fieldsText.update({'data': data})
        return fieldsText


# Parses last fields parameter (<time uint32><...>) as a single string
# Assumes it is formatted as <name number * type> or <name type>
# Returns: np.dtype
def parseFields(fieldstr):
    # Returns np.dtype from field string
    sep = re.split('\s', re.sub(r"\>\<|\>|\<", ' ', fieldstr).strip())
    # print(sep)
    typearr = []
    # Every two elmts is fieldname followed by datatype
    for i in range(0,sep.__len__(), 2):
        fieldname = sep[i]
        repeats = 1
        ftype = 'uint32'
        # Finds if a <num>* is included in datatype
        if sep[i+1].__contains__('*'):
            temptypes = re.split('\*', sep[i+1])
            # Results in the correct assignment, whether str is num*dtype or dtype*num
            ftype = temptypes[temptypes[0].isdigit()]
            repeats = int(temptypes[temptypes[1].isdigit()])
        else:
            ftype = sep[i+1]
        try:
            fieldtype = getattr(np, ftype)
        except AttributeError:
            print(ftype + " is not a valid field type.\n")
            exit(1)
        else:
            typearr.append((str(fieldname), fieldtype, repeats))
    return np.dtype(typearr)





In [50]:
""" New Method """
import numpy as np
os.chdir(r'C:\Users\dennis.tabuena\Desktop\_SPIKE_GADGETS\Trodes_2-5-1_Windows64\Trodes_2-5-1_Windows64\Resources\TrodesToPython')
from readTrodesExtractedDataFile3 import *



C:\Users\dennis.tabuena\Desktop\_SPIKE_GADGETS\Trodes_2-5-1_Windows64\Trodes_2-5-1_Windows64\Resources\TrodesToPython\readTrodesExtractedDataFile3.py:41: SyntaxWarning: invalid escape sequence '\s'
  sep = re.split('\s', re.sub(r"\>\<|\>|\<", ' ', fieldstr).strip())
C:\Users\dennis.tabuena\Desktop\_SPIKE_GADGETS\Trodes_2-5-1_Windows64\Trodes_2-5-1_Windows64\Resources\TrodesToPython\readTrodesExtractedDataFile3.py:51: SyntaxWarning: invalid escape sequence '\*'
  temptypes = re.split('\*', sep[i+1])


In [172]:
os.chdir(r'\\hive.gladstone.internal\Huang-LFP\TabuenaLFP\TRODE_EXPORTS\RAW_DATA\AngelfoodCake01.raw')
dat_cont = readTrodesExtractedDataFile('AngelfoodCake01.group0.coordinates.dat')

print(dat_cont)

{'description': 'Pad locations in microns', 'byte_order': 'little endian', 'original_file': 'AngelfoodCake01.rec', 'clockrate': '30000', 'trodes_version': '-1', 'compile_date': '-1', 'compile_time': '-1', 'qt_version': '-1', 'commit_tag': '-1', 'controller_firmware': '-1', 'headstage_firmware': '-1', 'controller_serialnum': '-1', 'headstage_serialnum': '-1', 'autosettle': '-1', 'smartref': '-1', 'gyro': '-1', 'accelerometer': '-1', 'magnetometer': '-1', 'time_offset': '0', 'system_time_at_creation': '-1', 'timestamp_at_creation': '-1', 'first_timestamp': '951222', 'fields': '<ml int32><dv int32><ap int32>', 'data': array([(-1, -1, -1), (-1, -1, -1), (-1, -1, -1), (-1, -1, -1),
       (-1, -1, -1), (-1, -1, -1), (-1, -1, -1), (-1, -1, -1),
       (-1, -1, -1), (-1, -1, -1), (-1, -1, -1), (-1, -1, -1),
       (-1, -1, -1), (-1, -1, -1), (-1, -1, -1), (-1, -1, -1),
       (-1, -1, -1), (-1, -1, -1), (-1, -1, -1), (-1, -1, -1),
       (-1, -1, -1), (-1, -1, -1), (-1, -1, -1), (-1, -1, -1),

C:\Users\dennis.tabuena\Desktop\_SPIKE_GADGETS\Trodes_2-5-1_Windows64\Trodes_2-5-1_Windows64\Resources\TrodesToPython\readTrodesExtractedDataFile3.py:64: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  return np.dtype(typearr)


In [179]:
os.chdir(r'\\hive.gladstone.internal\Huang-LFP\TabuenaLFP\Test_Emily\AngelfoodCake01')
os.listdir()
filename = "AngelfoodCake01.videoPositionTracking"
with open(filename, mode='rb') as file: # b is important -> binary
    filecontent = file.read()

In [ ]:
filecontent

In [217]:
format_dict = {'uint32':'I','uint16':'H'}
header = str(filecontent[:200])
print(header)
header = header.split('<End settings>')[0]
print(header_len)
field_section = header.split('Fields: ')[-1]
fields = re.findall(r'<.*?>',  field_section)
units = [f.split(' ')[-1][:-1] for f in fields]
field_names = [f.split(' ')[0][1:] for f in fields]
record_format = ''.join([format_dict[u] for u in units])

print(record_format)

b'<Start settings>\nthreshold: 56\ndark: 1\nFields: <time uint32><xloc uint16><yloc uint16><xloc2 uint16><yloc2 uint16>\n<End settings>\n\xb5\x83\x0e\x00\r\x01\x81\x01\x00\x00\x00\x00\xff\x8a\x0e\x00\x0c\x01\x81\x01\x00\x00\x00\x00\xff\x8a\x0e\x00\x0b\x01\x83\x01\x00\x00\x00\x00H\x92\x0e\x00\x0b\x01\x84\x01\x00\x00\x00\x00\xec\x95\x0e\x00\x0c\x01\x83\x01\x00\x00\x00\x005\x9d\x0e\x00\r\x01\x83\x01\x00\x00'
135
IHHHH


In [218]:
header_len = len(header+'<End settings>') - header.count('\\n')-1
record_format = ''.join([format_dict[u] for u in units])
record_size = struct.calcsize(record_format)
records = []
with open(filename, 'rb') as file:
    file.seek(header_len)
    while chunk := file.read(record_size):
        if len(chunk) == record_size:
            record = struct.unpack(record_format, chunk)
            records.append(record)
# display(records)
display(fields)
display(records[:10])
display(records[-10:])

['<time uint32>',
 '<xloc uint16>',
 '<yloc uint16>',
 '<xloc2 uint16>',
 '<yloc2 uint16>']

[(951221, 269, 385, 0, 0),
 (953087, 268, 385, 0, 0),
 (953087, 267, 387, 0, 0),
 (954952, 267, 388, 0, 0),
 (955884, 268, 387, 0, 0),
 (957749, 269, 387, 0, 0),
 (958682, 269, 388, 0, 0),
 (959615, 270, 388, 0, 0),
 (960547, 269, 388, 0, 0),
 (962412, 269, 389, 0, 0)]

[(132479231, 272, 388, 0, 0),
 (132479231, 272, 388, 0, 0),
 (132481096, 272, 388, 0, 0),
 (132482028, 272, 388, 0, 0),
 (132482961, 272, 387, 0, 0),
 (132484116, 272, 388, 0, 0),
 (132485759, 272, 387, 0, 0),
 (132486691, 272, 387, 0, 0),
 (132487624, 272, 387, 0, 0),
 (132489489, 272, 387, 0, 0)]

In [220]:
import pandas as pd
records_df = pd.DataFrame(records,columns = field_names)
records_df['time'] = records_df['time']/30000
#
display(records_df)
# records_df.to_csv('Export_pos.csv')

# # os.listdir()

,time,xloc,yloc,xloc2,yloc2
0,31.707367,269,385,0,0
1,31.769567,268,385,0,0
2,31.769567,267,387,0,0
3,31.831733,267,388,0,0
4,31.862800,268,387,0,0
...,...,...,...,...,...
109659,4416.137200,272,388,0,0
109660,4416.191967,272,387,0,0
109661,4416.223033,272,387,0,0
109662,4416.254133,272,387,0,0
